In [1]:
import pandas as pd
from sklearn import tree
from sklearn import metrics
import numpy as np
import graphviz 

In [25]:
df_match = pd.read_csv('../Project1/dataset.csv')
after_match_attributes = ['home_team_goal', 'away_team_goal', 'possession_home', 'shoton_home', 'shoton_away', 
                      'shotoff_home', 'shotoff_away', 'corner_home', 'corner_away', 'cross_home', 'cross_away', 
                      'foulcommit_home', 'foulcommit_away', 'rcard_home', 'rcard_away', 'ycard_home', 'ycard_away', 
                      'throwin_home', 'throwin_away']
training = df_match.sample(frac = 0.9)
training_args = training.drop(after_match_attributes, axis=1)
test = df_match[~df_match.isin(training)].dropna()
test_args = test.drop(after_match_attributes, axis=1)
home_win = training['home_team_goal'] > training['away_team_goal']

In [28]:
for max_depth in range(1, 25):
    clf = tree.DecisionTreeClassifier(max_depth = max_depth)
    clf.fit(training_args, home_win)
    predictions = clf.predict(test_args)
    actual = test['home_team_goal'] > test['away_team_goal']
    confusion = metrics.confusion_matrix(actual, predictions)
    #print(confusion)
    print (max_depth, ':\t', (confusion[0, 0] + confusion[1, 1]) / np.sum(confusion) * 100, '% correct')

1 :	 59.8326359833 % correct
2 :	 59.8326359833 % correct
3 :	 59.8326359833 % correct
4 :	 59.4142259414 % correct
5 :	 61.9246861925 % correct
6 :	 58.9958158996 % correct
7 :	 59.4142259414 % correct
8 :	 58.5774058577 % correct
9 :	 56.4853556485 % correct
10 :	 51.0460251046 % correct
11 :	 51.8828451883 % correct
12 :	 53.5564853556 % correct
13 :	 53.9748953975 % correct
14 :	 52.3012552301 % correct
15 :	 54.8117154812 % correct
16 :	 54.8117154812 % correct
17 :	 54.3933054393 % correct
18 :	 56.0669456067 % correct
19 :	 53.1380753138 % correct
20 :	 53.9748953975 % correct
21 :	 52.3012552301 % correct
22 :	 53.9748953975 % correct
23 :	 52.3012552301 % correct
24 :	 53.9748953975 % correct


In [11]:
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=training_args.columns)
graph = graphviz.Source(dot_data)
graph.render("home_win")

'home_win.pdf'